<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=1aa58d92faabb91bb36d9cc4b1528dab254c9001a8cab1fbe4360f99fd02cd78
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install yahoo_finance_api2

  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=d86acedfce9b98f8e1037ac48794db78221ce72d368ffc635065e2aae62b0a9f
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [49]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [50]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    #url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    #r = requests.get(url)
    #with open('data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [51]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [52]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [53]:
import datetime as dt
import os
import pytz

jst = pytz.timezone('Asia/Tokyo')
now = dt.datetime.now(jst)
now = now.strftime("%Y%m%d_%H%M%S")
# ディレクトリ名作成（YYYYMMDD_HHMMSS_screening）
directory = f'{now}_screening'

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [54]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


In [55]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, min_price=100, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

'timestamp'
銘柄コード[1954]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5128]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6924]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9810]の株価情報を取得できなかったため、スキップしました。
経過時間: 339.1479563713074秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1073100,678.000000
1,ＩＮＰＥＸ,1605,鉱業,9436600,1789.000000
2,オリエンタル白石,1786,建設業,589000,307.000000
3,大林組,1802,建設業,3110300,1299.000000
4,清水建設,1803,建設業,3928500,947.900024
...,...,...,...,...,...
280,大阪瓦斯,9532,電気・ガス業,990500,2173.500000
281,エイチ・アイ・エス,9603,サービス業,762700,2015.000000
282,エヌ・ティ・ティ・データ,9613,情報・通信業,1830800,1955.500000
283,ヤマダホールディングス,9831,小売業,2987500,430.700012


In [56]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1073100,678.000000
1,ＩＮＰＥＸ,1605,鉱業,9436600,1789.000000
2,大林組,1802,建設業,3110300,1299.000000
3,清水建設,1803,建設業,3928500,947.900024
4,長谷工コーポレーション,1808,建設業,617400,1827.000000
...,...,...,...,...,...
198,北海道電力,9509,電気・ガス業,646900,573.099976
199,電源開発,9513,電気・ガス業,943000,2207.500000
200,東京瓦斯,9531,電気・ガス業,913000,3186.000000
201,エイチ・アイ・エス,9603,サービス業,762700,2015.000000


In [57]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1073100,678.000000
1,ＩＮＰＥＸ,1605,鉱業,9436600,1789.000000
2,大林組,1802,建設業,3110300,1299.000000
3,清水建設,1803,建設業,3928500,947.900024
4,長谷工コーポレーション,1808,建設業,617400,1827.000000
...,...,...,...,...,...
107,日本テレビホールディングス,9404,情報・通信業,639100,1347.000000
108,ソフトバンク,9434,情報・通信業,4797900,1543.500000
109,四国電力,9507,電気・ガス業,684300,999.299988
110,電源開発,9513,電気・ガス業,943000,2207.500000


In [58]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [59]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,オリエンタル白石,1786,建設業,589000,307.000000
1,カルビー,2229,食料品,595300,2759.000000
2,明治ホールディングス,2269,食料品,655700,3366.000000
3,システナ,2317,情報・通信業,594000,290.000000
4,日本駐車場開発,2353,不動産業,2551500,219.000000
...,...,...,...,...,...
76,日本通信,9424,情報・通信業,1884200,231.000000
77,レノバ,9519,電気・ガス業,764200,1509.000000
78,大阪瓦斯,9532,電気・ガス業,990500,2173.500000
79,ヤマダホールディングス,9831,小売業,2987500,430.700012


In [60]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,オリエンタル白石,1786,建設業,589000,307.000000
1,日本駐車場開発,2353,不動産業,2551500,219.000000
2,キリンホールディングス,2503,食料品,2272100,2088.000000
3,北の達人コーポレーション,2930,化学,723600,264.000000
4,ビックカメラ,3048,小売業,831900,1070.000000
5,ＭｏｎｏｔａＲＯ,3064,小売業,1620000,1758.500000
6,Ｊ．フロント リテイリング,3086,小売業,1011400,1368.000000
7,ＺＯＺＯ,3092,小売業,1338500,2786.500000
8,トレジャー・ファクトリー,3093,小売業,777300,1427.000000
9,ウエルシアホールディングス,3141,小売業,630700,2727.000000


In [61]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '4_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [62]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4257,タキヒヨー,9982,卸売業
4260,蔵王産業,9986,卸売業
4265,ヤマザワ,9993,小売業
4266,やまや,9994,小売業


In [63]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 100円以上、5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_standard

'timestamp'
銘柄コード[1380]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1795]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1981]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[2293]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3322]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4327]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5918]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6265]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6392]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7425]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7500]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7515]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7531]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8006]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,250900,2813.0
1,技研ホールディングス,1443,建設業,315300,229.0
2,住石ホールディングス,1514,鉱業,483400,349.0
3,土屋ホールディングス,1840,建設業,1075200,263.0
4,大盛工業,1844,建設業,357000,218.0
...,...,...,...,...,...
66,トレイダーズホールディングス,8704,証券、商品先物取引業,286200,638.0
67,第一商品,8746,証券、商品先物取引業,273600,100.0
68,エリアクエスト,8912,不動産業,4557900,116.0
69,明豊エンタープライズ,8927,不動産業,336700,245.0


In [64]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,技研ホールディングス,1443,建設業,315300,229.000000
1,住石ホールディングス,1514,鉱業,483400,349.000000
2,土屋ホールディングス,1840,建設業,1075200,263.000000
3,大盛工業,1844,建設業,357000,218.000000
4,ＹＥ ＤＩＧＩＴＡＬ,2354,情報・通信業,242100,606.000000
5,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,1633100,717.000000
6,出前館,2484,情報・通信業,1170400,436.000000
7,ベクター,2656,小売業,269000,356.000000
8,イメージ ワン,2667,卸売業,252800,695.000000
9,アップルインターナショナル,2788,卸売業,207500,351.000000


In [65]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,土屋ホールディングス,1840,建設業,1075200,263.000000
1,大盛工業,1844,建設業,357000,218.000000
2,ＹＥ ＤＩＧＩＴＡＬ,2354,情報・通信業,242100,606.000000
3,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,1633100,717.000000
4,出前館,2484,情報・通信業,1170400,436.000000
5,ベクター,2656,小売業,269000,356.000000
6,アップルインターナショナル,2788,卸売業,207500,351.000000
7,クロスプラス,3320,卸売業,715400,1098.000000
8,クラウディアホールディングス,3607,繊維製品,2479800,929.000000
9,オルトプラス,3672,情報・通信業,381200,235.000000


In [66]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '2_東証S_UPO')

In [67]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,250900,2813.0
1,住石ホールディングス,1514,鉱業,483400,349.0
2,ｎｍｓ ホールディングス,2162,サービス業,252300,418.0
3,クシム,2345,情報・通信業,241000,307.0
4,プロパスト,3236,不動産業,205100,148.0
5,串カツ田中ホールディングス,3547,小売業,252800,1481.0
6,ｅｎｉｓｈ,3667,情報・通信業,486200,324.0
7,サイバーステップ,3810,情報・通信業,268000,401.0
8,リミックスポイント,3825,情報・通信業,517100,215.0
9,ウルトラファブリックス・ホールディングス,4235,化学,250700,2289.0


In [68]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,483400,349.0
1,ｎｍｓ ホールディングス,2162,サービス業,252300,418.0
2,クシム,2345,情報・通信業,241000,307.0
3,プロパスト,3236,不動産業,205100,148.0
4,串カツ田中ホールディングス,3547,小売業,252800,1481.0
5,ｅｎｉｓｈ,3667,情報・通信業,486200,324.0
6,サイバーステップ,3810,情報・通信業,268000,401.0
7,リミックスポイント,3825,情報・通信業,517100,215.0
8,ハーモニック・ドライブ・システムズ,6324,機械,309600,4075.0
9,リード,6982,輸送用機器,205700,735.0


In [69]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '5_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [70]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
55,ルーデン・ホールディングス,1400,建設業
56,エムビーエス,1401,建設業
65,Ｌｉｂ Ｗｏｒｋ,1431,建設業
70,フィット,1436,建設業
75,ニッソウ,1444,建設業
...,...,...,...
4079,プログリット,9560,サービス業
4080,グラッドキューブ,9561,サービス業
4081,ビジネスコーチ,9562,サービス業
4082,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [71]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3803]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4073]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7067]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,451500,1198.0
1,ウェッジホールディングス,2388,その他金融業,14599600,155.0
2,ジェイテック,2479,サービス業,291700,355.0
3,ベースフード,2936,食料品,1547900,528.0
4,海帆,3133,小売業,1769600,609.0
...,...,...,...,...,...
64,スマサポ,9342,サービス業,789400,1748.0
65,ｉｓｐａｃｅ,9348,サービス業,240000,1344.0
66,リニューアブル・ジャパン,9522,電気・ガス業,4577000,1041.0
67,Ｍ＆Ａ総研ホールディングス,9552,サービス業,496600,3590.0


In [72]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,451500,1198.0
1,ウェッジホールディングス,2388,その他金融業,14599600,155.0
2,ジェイテック,2479,サービス業,291700,355.0
3,ベースフード,2936,食料品,1547900,528.0
4,海帆,3133,小売業,1769600,609.0
5,パルマ,3461,不動産業,5397300,392.0
6,ティーケーピー,3479,不動産業,217200,2787.0
7,シェアリングテクノロジー,3989,情報・通信業,455900,592.0
8,アララ,4015,情報・通信業,496800,585.0
9,ティアンドエス,4055,情報・通信業,405100,3120.0


In [73]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ウェッジホールディングス,2388,その他金融業,14599600,155.0
1,ジェイテック,2479,サービス業,291700,355.0
2,ベースフード,2936,食料品,1547900,528.0
3,パルマ,3461,不動産業,5397300,392.0
4,アララ,4015,情報・通信業,496800,585.0
5,ティアンドエス,4055,情報・通信業,405100,3120.0
6,ＷＡＣＵＬ,4173,情報・通信業,206900,955.0
7,スペースマーケット,4487,情報・通信業,1117900,414.0
8,ペイロール,4489,情報・通信業,215600,1033.0
9,免疫生物研究所,4570,医薬品,1474000,660.0


In [74]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '3_東証G_UPO')

In [75]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,Ａｉｍｉｎｇ,3911,情報・通信業,895500,447.0
1,プラスアルファ・コンサルティング,4071,情報・通信業,454300,2680.0
2,ココナラ,4176,情報・通信業,595200,366.0
3,ＣＩＮＣ,4378,情報・通信業,949500,862.0
4,Ｃｈａｔｗｏｒｋ,4448,情報・通信業,226500,981.0
5,ＢＡＳＥ,4477,情報・通信業,1586900,243.0
6,フリー,4478,情報・通信業,325500,3145.0
7,アンジェス,4563,医薬品,1148100,104.0
8,キャンバス,4575,医薬品,381000,1077.0
9,デ・ウエスタン・セラピテクス研究所,4576,医薬品,3354300,222.0


In [76]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,プラスアルファ・コンサルティング,4071,情報・通信業,454300,2680.0
1,ＣＩＮＣ,4378,情報・通信業,949500,862.0
2,Ｃｈａｔｗｏｒｋ,4448,情報・通信業,226500,981.0
3,ＢＡＳＥ,4477,情報・通信業,1586900,243.0
4,フリー,4478,情報・通信業,325500,3145.0
5,アンジェス,4563,医薬品,1148100,104.0
6,キャンバス,4575,医薬品,381000,1077.0
7,サンバイオ,4592,医薬品,435800,591.0
8,ブライトパス・バイオ,4594,医薬品,275600,129.0
9,モダリス,4883,医薬品,401500,235.0


In [77]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [78]:
import pandas as pd

def remove_duplicate_rows(dataframe, *other_dataframes):
    """重複しているティッカーコードの行を削除する

    :param dataframe: 対象のデータフレーム
    :param other_dataframes: 重複しているかどうかの判断のためのデータフレーム
    :return 引数dataframeで重複行を削除したもの
    """
    # データフレームをコピーして新しいデータフレームを作成
    result_df = dataframe.copy()

    # 重複確認のためのセットを作成
    unique_values = set()
    for df in other_dataframes:
        unique_values.update(df['コード'])

    # 重複しているティッカーコードを削除
    result_df = result_df[~result_df['コード'].isin(unique_values)]

    return result_df

In [79]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[1380]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1795]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1981]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[2293]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3322]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4327]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5918]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6265]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6392]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7425]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7500]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7515]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7531]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8006]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘

,銘柄名,コード,業種,出来高,株価
0,土屋ホールディングス,1840,建設業,1075200,263.000000
1,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,1633100,717.000000
2,出前館,2484,情報・通信業,1170400,436.000000
3,クロスプラス,3320,卸売業,715400,1098.000000
4,クラウディアホールディングス,3607,繊維製品,2479800,929.000000
5,リミックスポイント,3825,情報・通信業,517100,215.000000
6,ネオマーケティング,4196,情報・通信業,1295400,1415.000000
7,Ｓｐｅｅｅ,4499,情報・通信業,528700,2849.000000
8,ノバシステム,5257,情報・通信業,530100,3005.000000
9,千代田化工建設,6366,建設業,754200,351.000000


In [80]:
# ティッカーコードが重複している行を削除
filtered_tse_standard_2_remove_duplicate = remove_duplicate_rows(filtered_tse_standard_2, filtered_perfect_order_week_day_tse_standard, filtered_reverse_perfect_order_week_day_tse_standard)
filtered_tse_standard_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
8,ノバシステム,5257,情報・通信業,530100,3005.0
9,千代田化工建設,6366,建設業,754200,351.0
11,ＪＡＬＣＯホールディングス,6625,不動産業,633500,202.0
14,フェローテックホールディングス,6890,電気機器,614600,3485.0
15,ヘリオス テクノ ホールディング,6927,電気機器,1443100,503.0
16,名村造船所,7014,輸送用機器,1546100,585.0
17,住信ＳＢＩネット銀行,7163,銀行業,500900,1596.0


In [81]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2_remove_duplicate, '7_東証S_選抜')

In [82]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows_2 = filtered_ticker(tse_grows, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_grows_2

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3803]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4073]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7067]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ウェッジホールディングス,2388,その他金融業,14599600,155.0
1,ベースフード,2936,食料品,1547900,528.0
2,海帆,3133,小売業,1769600,609.0
3,パルマ,3461,不動産業,5397300,392.0
4,Ａｉｍｉｎｇ,3911,情報・通信業,895500,447.0
5,プレイド,4165,情報・通信業,503000,811.0
6,ココナラ,4176,情報・通信業,595200,366.0
7,エクサウィザーズ,4259,情報・通信業,712600,474.0
8,ＣＩＮＣ,4378,情報・通信業,949500,862.0
9,ＢＡＳＥ,4477,情報・通信業,1586900,243.0


In [83]:
# ティッカーコードが重複している行を削除
filtered_tse_grows_2_remove_duplicate = remove_duplicate_rows(filtered_tse_grows_2, filtered_perfect_order_week_day_tse_grows, filtered_reverse_perfect_order_week_day_tse_grows)
filtered_tse_grows_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
2,海帆,3133,小売業,1769600,609.0
4,Ａｉｍｉｎｇ,3911,情報・通信業,895500,447.0
5,プレイド,4165,情報・通信業,503000,811.0
6,ココナラ,4176,情報・通信業,595200,366.0
7,エクサウィザーズ,4259,情報・通信業,712600,474.0
14,デ・ウエスタン・セラピテクス研究所,4576,医薬品,3354300,222.0
15,ヘリオス,4593,医薬品,739200,280.0
17,ティムス,4891,医薬品,995800,303.0
18,カバー,5253,情報・通信業,5342400,2460.0
19,モンスターラボホールディングス,5255,情報・通信業,1232700,709.0


In [84]:
# CSVファイル保存
save_df_as_csv(filtered_tse_grows_2_remove_duplicate, '8_東証G_選抜')